# Analysis of Israeli Restaurants in New York City 

### 1. Install all the libraries

In [1]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
import urllib, json

Get the NYC dataset from url

In [4]:
url = "https://cocl.us/new_york_dataset"
response = urllib.request.urlopen(url)
ny_data = json.loads(response.read().decode())
ny_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
ny_data=ny_data["features"]

In [6]:
ny_data

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

Turn json file into dataframe

In [7]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for data in ny_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
neighborhoods.shape

(306, 4)

In [11]:
latitude=40.7128
longitude=-74.0060
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
map_newyork

In [12]:
CLIENT_ID = 'ADGZI2EDUVQ5KWEC2CR3J4P1AFWGPFGMDEPPRWRLAWWYZMOW' # your Foursquare ID
CLIENT_SECRET = '5H3CTSKCDIY2UJTZUWDEEMD5WCAGMK30KS2AHO0XMF15VXMG' # your Foursquare Secret
VERSION = '20180605'

In [13]:
radius = 15000
limit=200
latitude=40.7128
longitude=-74.0060
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&query={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    "Israeli",
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ADGZI2EDUVQ5KWEC2CR3J4P1AFWGPFGMDEPPRWRLAWWYZMOW&client_secret=5H3CTSKCDIY2UJTZUWDEEMD5WCAGMK30KS2AHO0XMF15VXMG&query=Israeli&v=20180605&ll=40.7128,-74.006&radius=15000&limit=200'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d793097fd16bb002cf4d5ef'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'New York',
  'headerFullLocation': 'New York',
  'headerLocationGranularity': 'city',
  'query': 'israeli',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 40.847800135000135,
    'lng': -73.82822942705799},
   'sw': {'lat': 40.57779986499987, 'lng': -74.18377057294201}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f3046da7beb0cfa14dcac59',
       'name': 'Taïm Falafel and Smoothie Bar',
       'location': {'address': '45 Spring St',
        'crossStreet': 'at Mulberry St',
        'lat': 40.721936090419106,
        

In [15]:
venues = results['response']['groups'][0]['items']
israeli_venues=json_normalize(venues)
filtered_columns = ['venue.name', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
israeli_venues = israeli_venues.loc[:, filtered_columns]
israeli_venues=israeli_venues.rename(columns={"venue.name":"Name", "venue.location.postalCode":"Zipcode", "venue.location.lat":"Latitude", "venue.location.lng":"Longitude"})
israeli_venues.dropna(inplace=True)

Find and rename duplicates (when two locations exist with the same name)

In [16]:
dup = israeli_venues[israeli_venues.duplicated("Name")]
dup["Name"] = dup["Name"] + " 1"
dup

C:\Users\elama\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Name,Zipcode,Latitude,Longitude
13,12 Chairs 1,11249,40.714384,-73.965148
28,Jack's Wife Freda 1,10014,40.729999,-74.003584
31,Taïm 1,10010,40.742256,-73.993144
33,The Hummus & Pita Co 1,10011,40.739269,-73.995493
57,Hummus Kitchen 1,10028,40.776372,-73.952828


In [17]:
israeli_venues2 = israeli_venues.drop_duplicates("Name")
israeli_venues = israeli_venues2.append(dup)
israeli_venues = israeli_venues.reset_index(drop=True)

In [18]:
israeli_venues.tail()

,Name,Zipcode,Latitude,Longitude
84,12 Chairs 1,11249,40.714384,-73.965148
85,Jack's Wife Freda 1,10014,40.729999,-74.003584
86,Taïm 1,10010,40.742256,-73.993144
87,The Hummus & Pita Co 1,10011,40.739269,-73.995493
88,Hummus Kitchen 1,10028,40.776372,-73.952828


Map of New York with Israeli restaurants marked

### Sort restaurants by borough - assign borough according to zipcode

Get a list of zipcodes for each borough

In [19]:
zips=pd.read_html("https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm", index_col=None)
zips=zips[0]
zips=zips.drop("Neighborhood", axis=1)
zips = zips.groupby(['Borough'])['ZIP Codes'].apply(', '.join).reset_index()
zips.head()

,Borough,ZIP Codes
0,Bronx,"10453, 10457, 10460, 10458, 10467, 10468, 1045..."
1,Brooklyn,"11212, 11213, 11216, 11233, 11238, 11209, 1121..."
2,Manhattan,"10026, 10027, 10030, 10037, 10039, 10001, 1001..."
3,Queens,"11361, 11362, 11363, 11364, 11354, 11355, 1135..."
4,Staten Island,"10302, 10303, 10310, 10306, 10307, 10308, 1030..."


In [20]:
Bronx=zips.iat[0, 1]
Brooklyn=zips.iat[1, 1]
Manhattan=zips.iat[2, 1]
Queens=zips.iat[3, 1]
StatenIsland=zips.iat[4, 1]

Insert a "Borough" column

In [21]:
israeli_venues["Zipcode"].astype(str)
israeli_venues["Borough"]=""

Assign a borough to each venue according to its zipcode

In [22]:
index = 0
for venue in israeli_venues["Name"]:
    try:
        zipcode = israeli_venues.iat[index, 1]
    except IndexError:
        print("Something went wrong")
    index = israeli_venues[israeli_venues["Name"] == venue].index.values.astype(int)[0]
    zipcode = israeli_venues.iat[index, 1]
    if zipcode in Manhattan:
        israeli_venues.at[index,"Borough"]="Manhattan"
    elif zipcode in Brooklyn:
        israeli_venues.at[index,"Borough"]="Brooklyn"
    elif zipcode in Bronx:
        israeli_venues.at[index,"Borough"]="Bronx"
    elif zipcode in Queens:
        israeli_venues.at[index,"Borough"]="Queens"
    elif zipcode in StatenIsland:
        israeli_venues.at[index,"Borough"]="StatenIsland"
    else:
        israeli_venues.at[index,"Borough"]=None

Drop venues not assigned to any borough

In [23]:
israeli_venues = israeli_venues.dropna()

Visualize venues on NYC map

In [24]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(israeli_venues['Latitude'], israeli_venues['Longitude'], israeli_venues['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
map_newyork


In [25]:
israeli_venues.head()

,Name,Zipcode,Latitude,Longitude,Borough
0,Taïm Falafel and Smoothie Bar,10012,40.721936,-73.996279,Manhattan
1,12 Chairs,10012,40.727680,-74.002616,Manhattan
2,Miss Ada,11205,40.689560,-73.972351,Brooklyn
3,Nur,10003,40.738540,-73.988800,Manhattan
4,Spiegel,10009,40.723849,-73.987954,Manhattan
